Importation des librairies utilisées et des variables utiles

In [53]:
import unicodedata,string
import numpy as np
import pandas as pd
from collections import Counter
import os


#liste des 27 caractères "autorisés
letters = []
for x in string.ascii_lowercase+" ":
    letters.append(x)

## Etape 1

Traite les donnés brut .txt pour ne garder que 27 caractère (avec ou sans espaces)

In [54]:
#Ouvre un dossier (foldername) et concatène tous les fichiers .txt en un ficher .txt au nom de l'auteur
def concatennation(foldername, auteur):
    file_conca = open('textes/'+auteur+'_conca'+'.txt','a')
    for fichier in os.listdir(foldername):
        if fichier[-4:] == '.txt':
            with open(foldername + fichier,'r') as f:
                file_conca.write(f.read())
    file_conca.close()

#simplification d'un fichier .txt, enlève la ponctuation, les majuscules, les chiffres
#avec_espace permet de décider si on supprime aussi les espaces
def simplification(filename, avec_espaces):
    file1 = open(filename, 'r')
    data = file1.read()
    file1.close()
    
    #normalisation de l'encodage
    data = unicodedata.normalize('NFKD', data)
    
    #on supprime les majuscules
    data = data.lower()
    
    #remplace la ponctuation et les chiffres par des espaces
    dataBis = ""
    for x in data:
        if x in (string.punctuation+string.digits): 
            x = " "
        dataBis = dataBis + x
    
    #supprime les caractères spéciaux
    data = ''.join(x for x in dataBis if x in letters)
    
    #supprimme les espaces en trop et tous les espaces si avec_espaces = false
    if(avec_espaces):
        data = ' '.join(data.split())
        file2 = open(filename[:-4]+'_simplifie_avec_espaces.txt','w')
        file2.write(data)
    
    else:    
        data = ''.join(data.split())
        file2 = open(filename[:-4]+'_simplifie_sans_espaces.txt','w')
        file2.write(data)
    
    file2.close()

#return un string contenant l'ensemble d'un fichier .txt simplifié
#avec_espace permet de décider si on supprime aussi les espaces
def getData(filename, avec_espaces):
    simplification(filename, avec_espaces)
    if(avec_espaces):
        file = open(filename[:-4]+'_simplifie_avec_espaces.txt','r')
    else:
        file = open(filename[:-4]+'_simplifie_sans_espaces.txt','r')
    data = file.read()
    file.close()
    return data

## Etape 2

In [55]:
#on compte le nombre d'occurence de chacun des N_grams, résultat sous forme de dictionnaire
def N_grams(n, filename, avec_espaces):
    data=getData(filename, avec_espaces)
    
    #on met dans une liste l'ensemble des N_grams du fichier
    N_grams=[]
    for x in range(len(data)-n+1):
       N_grams.append(data[x:x+n])
    
    len(N_grams)
    ctn = Counter(N_grams)
    ctn['data_length'] = len(data)
    return ctn

#return la fréquence des des N_grams de taille n
def frequence_N_grams(n, filename, avec_espaces):
    ctn = N_grams(n,filename, avec_espaces)
    length = ctn.get('data_length')
    del ctn['data_length']
    
    if(avec_espaces):
        if(n==1): letters_n = letters
        if(n==2): letters_n = [x + y for x in letters for y in letters]
        if(n==3): letters_n = [x + y + z for x in letters for y in letters for z in letters]
        if(n==4): letters_n = [x + y + z + t for x in letters for y in letters for z in letters for t in letters]
        frequence = pd.Series(np.zeros(pow(27,n)) , index=letters_n)
        
    else:
        if(n==1): letters_n = letters[:26]
        if(n==2): letters_n = [x + y for x in letters[:26] for y in letters[:26]]
        if(n==3): letters_n = [x + y + z for x in letters[:26] for y in letters[:26] for z in letters[:26]]
        if(n==4): letters_n = [x + y + z + t for x in letters[:26] for y in letters[:26] for z in letters[:26] for t in letters[:26]]
        frequence = pd.Series(np.zeros(pow(26,n)) , index=letters_n)
    
    for key in ctn.keys():
        frequence.loc [key] = ctn.get(key)
        
    return frequence/length

#return la matrice de transition de mémoire n
def matrice_transition_memoire_n(n,filename):
    ctn = N_grams(n+1,filename, True)
    length = ctn.get('data_length')
    del ctn['data_length']
    
    #on calcul
    if(n==0): letters_n = ['frequence']
    if(n==1): letters_n = letters
    if(n==2): letters_n = [x + y for x in letters for y in letters]
    if(n==3): letters_n = [x + y + z for x in letters for y in letters for z in letters]
    if(n==4): letters_n = [x + y + z + t for x in letters for y in letters for z in letters for t in letters]
    
    result = pd.DataFrame(np.zeros((pow(27,n),27)), index=letters_n, columns=letters)
    if(n==0):
        for key in ctn.keys():
            result.loc['frequence',key[-1]] = ctn.get(key)
        
    
    else:
        for key in ctn.keys():
            result.loc[key[:-1],key[-1]] = ctn.get(key)
        
    return result/length


#return un texte généré aléatoirement à partir de la matrice de transition
#le debut du texte est décidé par l'utilisateur 
def generation_text_mem_N(longueur_text, matrice_transition, debut_text):
    text = debut_text
    
    #on récupère la mémoire de la matrice de transition
    memoire = int(np.log(matrice_transition.shape[0])/np.log(27))
    
    #on traite le cas sans mémoire à part
    if(memoire==0):
        for i in range (longueur_text):
            rand = np.random.uniform(0,1)
            sum=0
            for l in letters:
                if(sum <= rand < matrice_transition.loc['frequence',l] + sum):
                    text = text +l
                    break
                sum = matrice_transition.loc['frequence',l] + sum
                
    else:
        sup = np.sum(matrice_transition,1)
        for i in range (longueur_text):
            rand = np.random.uniform(0,sup.loc[text[-memoire:]])
            sum=0
            for l in letters:
                if(sum <= rand < matrice_transition.loc[text[-memoire:],l] + sum):
                    text = text +l
                    break
                sum = matrice_transition.loc[text[-memoire:],l] + sum
    return text



### Remarques

+ La création des 5 matrices de transition est un peu longue (~ 1 min sur mybinder.org)
+ On utilise ici la concaténation de 2 livres de Henry James (*The Golden Bowl* et *Glasses*), ce qui représente ~680 000 mots

In [56]:
%%time
m0 = matrice_transition_memoire_n(0,'textes/james_conca.txt')
m1 = matrice_transition_memoire_n(1,'textes/james_conca.txt')
m2 = matrice_transition_memoire_n(2,'textes/james_conca.txt')
m3 = matrice_transition_memoire_n(3,'textes/james_conca.txt')
m4 = matrice_transition_memoire_n(4,'textes/james_conca.txt')


CPU times: user 28.9 s, sys: 300 ms, total: 29.2 s
Wall time: 29.2 s


In [57]:
#On génère 5 textes aléatoires à partir des différentes matrices de transition
t0 = generation_text_mem_N(300, m0, '')
t1 = generation_text_mem_N(300, m1, 't')
t2 = generation_text_mem_N(300, m2, 'th')
t3 = generation_text_mem_N(300, m3, 'the')
t4 = generation_text_mem_N(300, m4, 'the ')

In [58]:
print('Texte mémoire 0 : \n \n' + t0)
print('\nTexte mémoire 1 : \n \n' + t1)
print('\nTexte mémoire 2 : \n \n' + t2)
print('\nTexte mémoire 3 : \n \n' + t3)
print('\nTexte mémoire 4 : \n \n' + t4)

Texte mémoire 0 : 
 
oirtslaea ahtb o eoi yielp iuamtiiaussc i le  io rroae st mtaheh otn   osene c hesieei h h   ir aiytdlo teaas  ghettghesoyacoot t dohaogoges niva  ewtatp fdter hhnttemltbdoo omr ai kbaar slg eymesteegttkit r epain ya wi asnssil hd etreoenntoh he esh at   tl aeaa crhr naordeswt iaadoa  htoa thfhliokt

Texte mémoire 1 : 
 
t ined e st n ig th oneathe pldne hap vedicad g ay tula y culofowheyoke an s has ive onker llay s as otldef whecthe osinemp e sor ly tiomy ee then p ahend hegghedinciesthtes thearlashe feresheprbeen owowhely tone ango aseve walin tomincr terache bylyimprr veweyoug iered shofof t arlingor hiot orer he

Texte mémoire 2 : 
 
thar darkhationg wit heate ouldur to more shey be disn toom he knot the wor faccand sper there she recusuffectiould mear to her pren but was id ittionst of thaved quardis had ingfectie to firculd finghtwer fir a toomed ady gaid and attiedly to gie bectableaughad unpeanias devercollo silearlowwhis ture

Texte mémoire 3 : 
 
these a

### Conclusion de l'étape 2

+ On voit bien que que plus on rajoute de mémoire plus on obtient de mots appartenants à la langue (anglais ici).

## Etape 3

In [59]:
infinity = float('inf')

#première fonction F proposé dans le sujet
def f1(x, data, frequence2, frequence4):
    d = frequence2.loc[data[x:x+2]]*frequence2.loc[data[x+2:x+4]]
    e = frequence4.loc[data[x:x+4]]
    if(d==0):
        return infinity
    elif(e==0):
        return -infinity
    else:
        return np.log(frequence4.loc[data[x:x+4]]/(frequence2.loc[data[x:x+2]]*frequence2.loc[data[x+2:x+4]]))

#deuxième fonction définie dans le sujet
def f2(x, data, frequence2, frequence3):
    d = frequence3.loc[data[x:x+3]]*frequence2.loc[data[x+3:x+5]]
    e = frequence2.loc[data[x:x+2]]*frequence3.loc[data[x+2:x+5]]
    if(d==0):
        return infinity
    elif(e==0):
        return -infinity
    else:
        return np.log((frequence2.loc[data[x:x+2]]*frequence3.loc[data[x+2:x+5]])/(frequence3.loc[data[x:x+3]]*frequence2.loc[data[x+3:x+5]]))

    
#rajoute un espace entre x+1 et x+2 si minimum local f1
def rajout_espaces_1(filename, frequence2 , frequence4):
    data = getData(filename, False)
    length = len(data)
    x = 1
    
    while(x<len(data)-5):
        if(x==1):
            a = f1(x-1, data, frequence2, frequence4)
            b = f1(x, data, frequence2, frequence4)
            c = f1(x+1, data, frequence2, frequence4)
            if(b<=c and b<a):
                data = data[:x+2] + ' ' + data[x+2:]
                max_avant = True
            else:
                max_avant = False
            
        else:
            if(max_avant):
                x+=1
                a = f1(x-1, data, frequence2, frequence4)
                b = f1(x, data, frequence2, frequence4)
                c = f1(x+1, data, frequence2, frequence4)
                if(b<=c and b<a):
                    data = data[:x+2] + ' ' + data[x+2:]
                    max_avant = True
                else:
                    max_avant = False
                
            else:
                a = b
                b = c
                c = f1(x+1, data, frequence2, frequence4)
                if(b<=c and b<a):
                    data = data[:x+2] + ' ' + data[x+2:]
                    max_avant = True
                else:
                    max_avant = False
        x+=1
    
    with open(filename[:-4] + '_rajout_espaces_1.txt','w') as f:
        f.write(data)
        
    return(data)

#rajoute un espace entre x+1 et x+2 si maximum local f2
def rajout_espaces_2(filename, frequence2 , frequence3):
    data = getData(filename, False)
    length = len(data)
    x = 1
    
    while(x<len(data)-6):
        if(x==1):
            a = f2(x-1, data, frequence2, frequence3)
            b = f2(x, data, frequence2, frequence3)
            c = f2(x+1, data, frequence2, frequence3)
            if(b>=c and b>a):
                data = data[:x+2] + ' ' + data[x+2:]
                max_avant = True
            else:
                max_avant = False
            
        else:
            if(max_avant):
                x+=1
                a = f2(x-1, data, frequence2, frequence3)
                b = f2(x, data, frequence2, frequence3)
                c = f2(x+1, data, frequence2, frequence3)
                if(b>=c and b>a):
                    data = data[:x+2] + ' ' + data[x+2:]
                    max_avant = True
                else:
                    max_avant = False
                
            else:
                a = b
                b = c
                c = f2(x+1, data, frequence2, frequence3)
                if(b>=c and b>a):
                    data = data[:x+2] + ' ' + data[x+2:]
                    max_avant = True
                else:
                    max_avant = False
        x+=1
    
    with open(filename[:-4] + '_rajout_espaces_2.txt','w') as f:
        f.write(data)
        
    return(data)

#rajoute un espace entre x+1 et x+2 si f1 est minimum local et f2 maximum local
def rajout_espaces_1_2(filename, frequence2 , frequence3, frequence4):
    data = getData(filename, False)
    length = len(data)
    x = 1
    
    while(x<len(data)-6):
        if(x==1):
            a = f1(x-1, data, frequence2, frequence4)
            b = f1(x, data, frequence2, frequence4)
            c = f1(x+1, data, frequence2, frequence4)
            d = f2(x-1, data, frequence2, frequence3)
            e = f2(x, data, frequence2, frequence3)
            f = f2(x+1, data, frequence2, frequence3)
            
            if(b<=c and b<a and e>d and e>=f):
                data = data[:x+2] + ' ' + data[x+2:]
                max_avant = True
            else:
                max_avant = False
            
        else:
            if(max_avant):
                x+=1
                a = f1(x-1, data, frequence2, frequence4)
                b = f1(x, data, frequence2, frequence4)
                c = f1(x+1, data, frequence2, frequence4)
                d = f2(x-1, data, frequence2, frequence3)
                e = f2(x, data, frequence2, frequence3)
                f = f2(x+1, data, frequence2, frequence3)
                if(b<=c and b<a and e>d and e>=f):
                    data = data[:x+2] + ' ' + data[x+2:]
                    max_avant = True
                else:
                    max_avant = False
                
            else:
                a = b
                b = c
                c = f1(x+1, data, frequence2, frequence4)
                d = e
                e = f
                f = f2(x+1, data, frequence2, frequence3)
                if(b<=c and b<a and e>d and e>=f):
                    data = data[:x+2] + ' ' + data[x+2:]
                    max_avant = True
                else:
                    max_avant = False
        x+=1
    
    with open(filename[:-4] + '_rajout_espaces_1_2.txt','w') as f:
        f.write(data)
        
    return(data)

In [60]:
%%time
frequence2 = frequence_N_grams(2, 'textes/james_conca.txt', True)
frequence3 = frequence_N_grams(3, 'textes/james_conca.txt', True)
frequence4 = frequence_N_grams(4, 'textes/james_conca.txt', True)


CPU times: user 10.1 s, sys: 220 ms, total: 10.3 s
Wall time: 10.3 s


On teste sur un petit fichier texte venant d'un livre de Henry James (rajouter des espaces sur tout le livre est très long)

In [61]:
p = rajout_espaces_1_2('textes/test.txt', frequence2, frequence3, frequence4)
c = rajout_espaces_1('textes/test.txt', frequence2,frequence4)
k = rajout_espaces_2('textes/test.txt', frequence2,frequence3)

In [62]:
print('Rajout d\'espace avec la fonction 1 : \n \n' + c + '\n')
print('Rajout d\'espace avec la fonction 2 : \n \n' + k + '\n')
print('Rajout d\'espace avec les fonctions 1 et 2 : \n \n' + p + '\n')

Rajout d'espace avec la fonction 1 : 
 
the gold e nbow lvolum e i h enr yjame s book first the pr ince part firsti the pr ince had al ways liked his lo ndo nwhe n i th a dco met o him he wa s oneo fthem o d e rnroma nswho f in dbythe th ame sa m o re conv inc i ngimageo fthe trutho fthea ncient st a t e than anyt h e yhave lef tbythet i be rbrou ght upo nthe le ge ndof thec i tytow h ich the w orl dpaid t ribut e her eco gnised in the pre sentl o ndon much more tha n in con tem p o rary ro me ther e a ldimensionso fsuch ac as e ifi twas a qu estiono fa n imp eriu mh e saidt o him selfa ndi fo ne w i sheda sa roman t o re co veral i ttle the senseo fthat the placet o dos o wa s on lo ndon brid ge o re ve non a f in e after noo ni nmaya t hyde par kcor ne ri twas noti ndeedt oeit hero fthose places that thes e gr o undso fhis pr ed i l e ction after alls uffic ientl yvagu ehad at the mom ent w ear econc erned with him gui d e dhiss t epshe had straye dsimpl y en ough into b o ndst r eet

### Conclusion de l'étape 2

+ Les fonctions 1 et 2 utilisées de manière séparée semblent mettre trop d'espaces
+ La combinaison des 2 semblent elle ne pas ajouter suffisament d'espaces